In [2]:
# import modules
import pandas as pd
import numpy as np
import sqlalchemy as sqla
import pickle
from sqlalchemy import create_engine

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import pymysql
from sshtunnel import SSHTunnelForwarder
from datetime import datetime, timedelta

In [3]:
ssh_host = '137.43.49.79'
ssh_port = 22
ssh_username = 'student'
ssh_password = 'Ucd-cs-2023!'

mysql_user = 'root'
mysql_password = 'Group@18'
mysql_db = 'smartcityexplorer'

with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_password=ssh_password,
    remote_bind_address=('127.0.0.1', 3306)
) as tunnel:
    conn = pymysql.connect(
        host='127.0.0.1',
        user=mysql_user,
        password=mysql_password,
        db=mysql_db,
        port=tunnel.local_bind_port
    )
    
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_merged")
    results = cursor.fetchall()

    # Convert the results to a pandas DataFrame
    import pandas as pd
    df_venue_merged = pd.DataFrame(results, columns=[column[0] for column in cursor.description])

    # Close the database connection
    conn.close()

In [4]:
df_venue_merged

,venue_id,venue_name,busyness,venue_type,venue_address,merged_time,temperature_2m,apparent_temperature,precipitation,weathercode,visibility,windspeed_10m
0,47508476,Han Dynasty Upper West Side,0,CHINESE_RESTAURANT,"215 W 85th St New York, NY 10024 United States",2023-06-19 06:00:00,19.1,19.1,0.0,0,24900.0,6.1
1,47508476,Han Dynasty Upper West Side,0,CHINESE_RESTAURANT,"215 W 85th St New York, NY 10024 United States",2023-06-19 07:00:00,18.8,18.7,0.0,0,24600.0,6.6
2,47508476,Han Dynasty Upper West Side,0,CHINESE_RESTAURANT,"215 W 85th St New York, NY 10024 United States",2023-06-19 08:00:00,18.5,18.5,0.0,0,23200.0,6.1
3,47508476,Han Dynasty Upper West Side,0,CHINESE_RESTAURANT,"215 W 85th St New York, NY 10024 United States",2023-06-19 09:00:00,17.9,18.1,0.0,0,20200.0,6.6
4,47508476,Han Dynasty Upper West Side,0,CHINESE_RESTAURANT,"215 W 85th St New York, NY 10024 United States",2023-06-19 10:00:00,18.0,18.0,0.0,2,22400.0,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...
231949,31643001,Hallett Nature Sanctuary,0,NATURE_RESERVE,"6th Avenue & Central Park S New York, NY 10019...",2023-07-10 19:00:00,27.9,30.0,0.0,3,27600.0,16.3
231950,31643001,Hallett Nature Sanctuary,0,NATURE_RESERVE,"6th Avenue & Central Park S New York, NY 10019...",2023-07-10 20:00:00,28.2,29.6,0.0,3,28100.0,16.5
231951,31643001,Hallett Nature Sanctuary,0,NATURE_RESERVE,"6th Avenue & Central Park S New York, NY 10019...",2023-07-10 21:00:00,28.2,29.0,0.0,3,30400.0,17.7
231952,31643001,Hallett Nature Sanctuary,0,NATURE_RESERVE,"6th Avenue & Central Park S New York, NY 10019...",2023-07-10 22:00:00,28.2,28.6,0.0,0,33100.0,16.8


In [5]:
df_venue_merged.drop(['venue_name','venue_type', 'venue_address'], axis=1, inplace=True)

In [6]:
df_venue_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231954 entries, 0 to 231953
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   venue_id              231954 non-null  object        
 1   busyness              231954 non-null  int64         
 2   merged_time           231954 non-null  datetime64[ns]
 3   temperature_2m        231954 non-null  float64       
 4   apparent_temperature  231954 non-null  float64       
 5   precipitation         231954 non-null  float64       
 6   weathercode           231954 non-null  int64         
 7   visibility            231954 non-null  float64       
 8   windspeed_10m         231954 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(2), object(1)
memory usage: 15.9+ MB


In [7]:
# look for duplicates
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_merged.duplicated().sum())

# use "keep=False" to mark all duplicates as true, including the original rows that were duplicated
print('Number of duplicate rows (including first) in the table is:', df_venue_merged[df_venue_merged.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  40344
Number of duplicate rows (including first) in the table is: 64552


In [8]:
df_venue_merged = df_venue_merged.drop_duplicates()

In [9]:
# check rows after removing duplicates
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_merged.duplicated().sum())
df_venue_merged

Number of duplicate (excluding first) rows in the table is:  0


,venue_id,busyness,merged_time,temperature_2m,apparent_temperature,precipitation,weathercode,visibility,windspeed_10m
0,47508476,0,2023-06-19 06:00:00,19.1,19.1,0.0,0,24900.0,6.1
1,47508476,0,2023-06-19 07:00:00,18.8,18.7,0.0,0,24600.0,6.6
2,47508476,0,2023-06-19 08:00:00,18.5,18.5,0.0,0,23200.0,6.1
3,47508476,0,2023-06-19 09:00:00,17.9,18.1,0.0,0,20200.0,6.6
4,47508476,0,2023-06-19 10:00:00,18.0,18.0,0.0,2,22400.0,6.2
...,...,...,...,...,...,...,...,...,...
231949,31643001,0,2023-07-10 19:00:00,27.9,30.0,0.0,3,27600.0,16.3
231950,31643001,0,2023-07-10 20:00:00,28.2,29.6,0.0,3,28100.0,16.5
231951,31643001,0,2023-07-10 21:00:00,28.2,29.0,0.0,3,30400.0,17.7
231952,31643001,0,2023-07-10 22:00:00,28.2,28.6,0.0,0,33100.0,16.8


In [10]:
# convert last_update to date-time
df_venue_merged['merged_time'] = pd.to_datetime(df_venue_merged['merged_time'], unit='ms')

In [11]:
df_venue_merged

,venue_id,busyness,merged_time,temperature_2m,apparent_temperature,precipitation,weathercode,visibility,windspeed_10m
0,47508476,0,2023-06-19 06:00:00,19.1,19.1,0.0,0,24900.0,6.1
1,47508476,0,2023-06-19 07:00:00,18.8,18.7,0.0,0,24600.0,6.6
2,47508476,0,2023-06-19 08:00:00,18.5,18.5,0.0,0,23200.0,6.1
3,47508476,0,2023-06-19 09:00:00,17.9,18.1,0.0,0,20200.0,6.6
4,47508476,0,2023-06-19 10:00:00,18.0,18.0,0.0,2,22400.0,6.2
...,...,...,...,...,...,...,...,...,...
231949,31643001,0,2023-07-10 19:00:00,27.9,30.0,0.0,3,27600.0,16.3
231950,31643001,0,2023-07-10 20:00:00,28.2,29.6,0.0,3,28100.0,16.5
231951,31643001,0,2023-07-10 21:00:00,28.2,29.0,0.0,3,30400.0,17.7
231952,31643001,0,2023-07-10 22:00:00,28.2,28.6,0.0,0,33100.0,16.8


In [12]:
df_venue_merged['time'] = df_venue_merged['merged_time'].dt.strftime('%Y-%m-%d %H')
df_venue_merged['hour'] = df_venue_merged['merged_time'].dt.strftime('%H')
df_venue_merged['weekday']= df_venue_merged['merged_time'].dt.weekday
df_venue_merged

,venue_id,busyness,merged_time,temperature_2m,apparent_temperature,precipitation,weathercode,visibility,windspeed_10m,time,hour,weekday
0,47508476,0,2023-06-19 06:00:00,19.1,19.1,0.0,0,24900.0,6.1,2023-06-19 06,06,0
1,47508476,0,2023-06-19 07:00:00,18.8,18.7,0.0,0,24600.0,6.6,2023-06-19 07,07,0
2,47508476,0,2023-06-19 08:00:00,18.5,18.5,0.0,0,23200.0,6.1,2023-06-19 08,08,0
3,47508476,0,2023-06-19 09:00:00,17.9,18.1,0.0,0,20200.0,6.6,2023-06-19 09,09,0
4,47508476,0,2023-06-19 10:00:00,18.0,18.0,0.0,2,22400.0,6.2,2023-06-19 10,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
231949,31643001,0,2023-07-10 19:00:00,27.9,30.0,0.0,3,27600.0,16.3,2023-07-10 19,19,0
231950,31643001,0,2023-07-10 20:00:00,28.2,29.6,0.0,3,28100.0,16.5,2023-07-10 20,20,0
231951,31643001,0,2023-07-10 21:00:00,28.2,29.0,0.0,3,30400.0,17.7,2023-07-10 21,21,0
231952,31643001,0,2023-07-10 22:00:00,28.2,28.6,0.0,0,33100.0,16.8,2023-07-10 22,22,0


In [13]:
 df_venue_merged['weekday'].value_counts()

0    39690
1    25320
2    25320
3    25320
4    25320
5    25320
6    25320
Name: weekday, dtype: int64

In [14]:
hour_mean = df_venue_merged.groupby(['venue_id','time','hour','weekday','temperature_2m', 'apparent_temperature','precipitation','weathercode','visibility','windspeed_10m'])['busyness'].mean().reset_index()
print(hour_mean)

        venue_id           time hour  weekday  temperature_2m  \
0       10162275  2023-06-19 06   06        0            19.1   
1       10162275  2023-06-19 07   07        0            18.8   
2       10162275  2023-06-19 08   08        0            18.5   
3       10162275  2023-06-19 09   09        0            17.9   
4       10162275  2023-06-19 10   10        0            18.0   
...          ...            ...  ...      ...             ...   
191605  99797939  2023-07-10 19   19        0            27.9   
191606  99797939  2023-07-10 20   20        0            28.2   
191607  99797939  2023-07-10 21   21        0            28.2   
191608  99797939  2023-07-10 22   22        0            28.2   
191609  99797939  2023-07-10 23   23        0            27.7   

        apparent_temperature  precipitation  weathercode  visibility  \
0                       19.1            0.0            0     24900.0   
1                       18.7            0.0            0     24600.0   
2  

In [15]:
df_venueinfo = pd.DataFrame(hour_mean.to_records())
df_venueinfo

,index,venue_id,time,hour,weekday,temperature_2m,apparent_temperature,precipitation,weathercode,visibility,windspeed_10m,busyness
0,0,10162275,2023-06-19 06,06,0,19.1,19.1,0.0,0,24900.0,6.1,0.0
1,1,10162275,2023-06-19 07,07,0,18.8,18.7,0.0,0,24600.0,6.6,5.0
2,2,10162275,2023-06-19 08,08,0,18.5,18.5,0.0,0,23200.0,6.1,5.0
3,3,10162275,2023-06-19 09,09,0,17.9,18.1,0.0,0,20200.0,6.6,5.0
4,4,10162275,2023-06-19 10,10,0,18.0,18.0,0.0,2,22400.0,6.2,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...
191605,191605,99797939,2023-07-10 19,19,0,27.9,30.0,0.0,3,27600.0,16.3,55.0
191606,191606,99797939,2023-07-10 20,20,0,28.2,29.6,0.0,3,28100.0,16.5,50.0
191607,191607,99797939,2023-07-10 21,21,0,28.2,29.0,0.0,3,30400.0,17.7,35.0
191608,191608,99797939,2023-07-10 22,22,0,28.2,28.6,0.0,0,33100.0,16.8,0.0


In [16]:
# save cleaned dataframe in csv file
df_venueinfo.to_csv('venue_cleaned_busyness_weather.csv', index=False)

In [17]:
df_venueinfo.dtypes

index                     int64
venue_id                 object
time                     object
hour                     object
weekday                   int64
temperature_2m          float64
apparent_temperature    float64
precipitation           float64
weathercode               int64
visibility              float64
windspeed_10m           float64
busyness                float64
dtype: object

In [18]:
df_venueinfo['hour'] = df_venueinfo['hour'].astype('int64')
df_venueinfo['venue_id'] = df_venueinfo['venue_id'].astype('int64')

In [19]:
df_venueinfo.dtypes

index                     int64
venue_id                  int64
time                     object
hour                      int64
weekday                   int64
temperature_2m          float64
apparent_temperature    float64
precipitation           float64
weathercode               int64
visibility              float64
windspeed_10m           float64
busyness                float64
dtype: object

In [20]:
df_venueinfo['venue_id'].nunique()

356

In [21]:
df_venueinfo.corr()

,index,venue_id,hour,weekday,temperature_2m,apparent_temperature,precipitation,weathercode,visibility,windspeed_10m,busyness
index,1.000000,0.792039,0.000104,0.000382,0.001976,0.002216,-0.001089,0.000532,0.000173,-0.000676,0.028030
venue_id,0.792039,1.000000,0.000006,-0.000057,-0.000075,-0.000081,0.000091,0.000022,-0.000042,0.000094,0.035678
hour,0.000104,0.000006,1.000000,-0.023029,0.455115,0.356682,0.022762,0.165195,0.358751,0.396363,0.386889
weekday,0.000382,-0.000057,-0.023029,1.000000,0.071992,0.081122,0.099491,-0.002838,-0.010933,-0.057289,0.053802
temperature_2m,0.001976,-0.000075,0.455115,0.071992,1.000000,0.953256,-0.188897,0.224871,0.563396,0.071752,0.217229
apparent_temperature,0.002216,-0.000081,0.356682,0.081122,0.953256,1.000000,-0.143510,0.258765,0.334504,-0.153306,0.206579
precipitation,-0.001089,0.000091,0.022762,0.099491,-0.188897,-0.143510,1.000000,-0.020375,-0.267034,0.015667,-0.004523
weathercode,0.000532,0.000022,0.165195,-0.002838,0.224871,0.258765,-0.020375,1.000000,-0.047200,0.030375,0.120160
visibility,0.000173,-0.000042,0.358751,-0.010933,0.563396,0.334504,-0.267034,-0.047200,1.000000,0.329547,0.119830
windspeed_10m,-0.000676,0.000094,0.396363,-0.057289,0.071752,-0.153306,0.015667,0.030375,0.329547,1.000000,0.127516


In [22]:
df_venueinfo.to_csv('training_data.csv', index=False)

In [23]:
df_train=pd.read_csv('training_data.csv');
df_train.sort_values(['venue_id','time']).head(20)
df_train.shape

(191610, 12)

# ONE MODEL FOR ALL THE Venue

In [24]:
def printMetrics(testActualVal, predictions):
    #classification evaluation measures
    print('==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(testActualVal, predictions))
    #print("MSE: ", metrics.mean_squared_error(testActualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(testActualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(testActualVal, predictions))

In [25]:
train_feature = ['venue_id','hour', 'weekday','temperature_2m','apparent_temperature','precipitation','weathercode','visibility','windspeed_10m']
target_feature1 = ['busyness']

train=df_train[train_feature]
target=df_train[target_feature1]

X_train, X_test, y_train, y_test  = train_test_split(train,target,test_size=0.3)
X_train

,venue_id,hour,weekday,temperature_2m,apparent_temperature,precipitation,weathercode,visibility,windspeed_10m
42065,28333784,23,4,21.0,23.4,86.0,3,13100.0,8.3
25462,20962841,15,1,26.1,31.0,0.0,95,16700.0,3.6
187375,9877144,11,5,20.0,23.0,96.0,3,3200.0,3.4
40703,2779193,9,1,23.1,27.2,0.0,0,14300.0,2.1
10184,1600094,21,0,30.9,34.1,0.0,96,27500.0,14.7
...,...,...,...,...,...,...,...,...,...
77697,45920957,3,0,22.7,25.4,0.0,3,12200.0,15.3
44030,29244447,18,3,32.9,35.2,0.0,1,45100.0,18.4
54299,34227583,22,0,30.5,34.1,0.0,96,25000.0,13.6
52319,33357157,12,1,18.3,18.4,0.0,3,16400.0,10.7


In [ ]:
## Linear 

In [26]:
clf = LinearRegression()
clf = clf.fit(X_train,y_train)
score = clf.score(X_test,y_test)
score

0.1854283891390306

In [27]:
test_prediction = clf.predict(X_test)
printMetrics(y_test, test_prediction)

MAE:  19.927536438572123
RMSE:  25.511228385244184
R2:  0.1854283891390306


## Random Forest

In [28]:
rfc = RandomForestRegressor(random_state=0)
rfc = rfc.fit(X_train,y_train)
score = rfc.score(X_test,y_test)
test_prediction = rfc.predict(X_test)
printMetrics(y_test, test_prediction)

MAE:  5.316126506967277
RMSE:  10.023411997741826
R2:  0.8742529827645545


In [29]:
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = train_feature,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
hour,0.454488
venue_id,0.439813
weekday,0.038428
windspeed_10m,0.016878
visibility,0.015753
temperature_2m,0.012663
apparent_temperature,0.012398
weathercode,0.005784
precipitation,0.003797


## XGBOOST

In [43]:
xgboost_model = XGBRegressor(random_state=0)
xgboost_model = xgboost_model.fit(X_train, y_train)
score = xgboost_model.score(X_test, y_test)
test_prediction = xgboost_model.predict(X_test)
printMetrics(y_test, test_prediction)


MAE:  2.262869395345106
RMSE:  4.4098685066477765
R2:  0.9621477734883933


# MODEL FOR EACH Venue

## Linear Regression

In [30]:
model_sets = {}
venues = df_train['venue_id'].unique()
venues

array([10162275,  1030056, 10806948, 10807076, 11285867, 11342059,
       12087255, 12162796, 12614856, 13615083, 14030565, 14231569,
        1438228, 14983708, 15210835, 15730101, 15828162, 15899557,
        1600094, 16043955, 16397959, 17172582, 17384094, 17462102,
        1748305, 17804860, 17902456, 17961872,  1828839, 18484315,
       18698787, 19008253, 19070061, 19253272, 19346517, 19687702,
       19697447, 20119380, 20267807, 20574086, 20614058, 20648115,
       20676486,  2068527, 20848781,  2093009, 20962841, 21612100,
       21844013, 21868851, 21945075, 21975768, 22338592, 22721619,
       23357761, 23424895, 23432873, 23844278, 23854076, 23999761,
        2412410,  2440176, 24707214, 25338191, 25433436, 25540736,
       26206424,  2659367, 26616140, 27144611,  2714653, 27147400,
        2760359, 27660618, 27787334,  2779193, 28080527, 28207368,
       28333784, 28402389, 29199346, 29244447, 29509676, 29644604,
       29649961, 29929585,     3031, 30429096, 30764981, 31013

In [35]:
for venue in venues:
    
    # select one station
    venue_data = df_train[df_train['venue_id'] == venue]
    
  # linear regression feature
    X = venue_data[['hour', 'weekday','temperature_2m','apparent_temperature','precipitation','weathercode','visibility','windspeed_10m']]
    
    y = venue_data['busyness']
    
    # split into train and test data
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    # apply LinearRegression
    model = LinearRegression()
    
    # fit model
    model.fit(X_train, y_train)
    # put into dictionary
    model_sets[venue] = model
    
    
    test_prediction = model.predict(X_test)
    print('\n==============================================================================')
    print("venue: ",venue," LinearRegression ")
    printMetrics(y_test, test_prediction)


venue:  10162275  LinearRegression 
MAE:  19.33117291325937
RMSE:  24.069264978996525
R2:  0.35250486487325394

venue:  1030056  LinearRegression 
MAE:  15.672337619547845
RMSE:  19.71970220111052
R2:  0.20964259965817844

venue:  10806948  LinearRegression 
MAE:  12.44317735103475
RMSE:  17.107957552402738
R2:  0.4445712376106321

venue:  10807076  LinearRegression 
MAE:  13.841388442299504
RMSE:  18.609457743601347
R2:  0.682111971530815

venue:  11285867  LinearRegression 
MAE:  23.69020129757205
RMSE:  30.573626235797033
R2:  0.11079394648297036

venue:  11342059  LinearRegression 
MAE:  11.790701737652666
RMSE:  17.482952196995928
R2:  0.18526295842662432

venue:  12087255  LinearRegression 
MAE:  18.45544133787507
RMSE:  22.793633678500324
R2:  0.2341061596423758

venue:  12162796  LinearRegression 
MAE:  19.79633664689375
RMSE:  23.564800245036313
R2:  0.3067411337393776

venue:  12614856  LinearRegression 
MAE:  13.023981762727159
RMSE:  18.312663612201604
R2:  0.4322563920193

## Random Forest

In [38]:
model_sets = {}

for venue in venues:
    # select one station
    venue_data = df_train[df_train['venue_id'] == venue]

    # Random Forest Regression features
    X = venue_data[['hour', 'weekday', 'temperature_2m', 'apparent_temperature', 'precipitation', 'weathercode', 'visibility', 'windspeed_10m']]
    y = venue_data['busyness']

    # split into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    # apply Random Forest Regression
    model = RandomForestRegressor()

    # fit model
    model.fit(X_train, y_train)
    # put into dictionary
    model_sets[venue] = model

    test_prediction = model.predict(X_test)
    print('\n==============================================================================')
    print("venue:", venue, "Random Forest Regression")
    printMetrics(y_test, test_prediction)



venue: 10162275 Random Forest Regression
MAE:  2.1804878048780485
RMSE:  4.220081203703746
R2:  0.9800954451578051

venue: 1030056 Random Forest Regression
MAE:  2.403048780487805
RMSE:  4.570988178373389
R2:  0.9575338376205302

venue: 10806948 Random Forest Regression
MAE:  2.261280487804878
RMSE:  5.825790274037778
R2:  0.935591715821856

venue: 10807076 Random Forest Regression
MAE:  4.260060975609757
RMSE:  7.599383198655006
R2:  0.9469893139801167

venue: 11285867 Random Forest Regression
MAE:  2.736280487804878
RMSE:  6.2481326478689505
R2:  0.962862814351614

venue: 11342059 Random Forest Regression
MAE:  1.6204268292682926
RMSE:  5.642525543778701
R2:  0.9151337545341024

venue: 12087255 Random Forest Regression
MAE:  2.8204268292682926
RMSE:  5.862880597661786
R2:  0.9493285249190193

venue: 12162796 Random Forest Regression
MAE:  2.0259146341463414
RMSE:  3.018451892819942
R2:  0.9886253923715429

venue: 12614856 Random Forest Regression
MAE:  1.9246951219512194
RMSE:  4.26

## XGBOOST

In [42]:


model_sets = {}

for venue in venues:
    # select one station
    venue_data = df_train[df_train['venue_id'] == venue]

    # XGBoost Regression features
    X = venue_data[['hour', 'weekday', 'temperature_2m', 'apparent_temperature', 'precipitation', 'weathercode', 'visibility', 'windspeed_10m']]
    y = venue_data['busyness']

    # split into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    # apply XGBoost Regression
    model = XGBRegressor()

    # fit model
    model.fit(X_train, y_train)
    # put into dictionary
    model_sets[venue] = model

    test_prediction = model.predict(X_test)
    print('\n==============================================================================')
    print("venue:", venue, "XGBoost Regression")
    printMetrics(y_test, test_prediction)



venue: 10162275 XGBoost Regression
MAE:  2.129009478092307
RMSE:  3.740454865286537
R2:  0.984362774796513

venue: 1030056 XGBoost Regression
MAE:  2.1873444607648174
RMSE:  3.7014506030927667
R2:  0.972153748850642

venue: 10806948 XGBoost Regression
MAE:  1.2062346285614385
RMSE:  2.4654525422350355
R2:  0.9884647946081506

venue: 10807076 XGBoost Regression
MAE:  3.814204577616382
RMSE:  6.806923824313691
R2:  0.9574687137730428

venue: 11285867 XGBoost Regression
MAE:  2.920393132635182
RMSE:  6.408892503965452
R2:  0.9609272047912705

venue: 11342059 XGBoost Regression
MAE:  1.6756485899972087
RMSE:  4.6503324483174975
R2:  0.942355759132514

venue: 12087255 XGBoost Regression
MAE:  2.542380090514709
RMSE:  4.976785952302412
R2:  0.9634876847059883

venue: 12162796 XGBoost Regression
MAE:  1.7827490903735275
RMSE:  2.61014553382197
R2:  0.9914945488780755

venue: 12614856 XGBoost Regression
MAE:  1.3547428027826445
RMSE:  3.351252224580122
R2:  0.9809864396206824

venue: 13615083

# Ignore Below

In [1]:
import json

# Read the JSON file
with open('shopping_venueid_response4.json') as file:
    json_data = json.load(file)

# Specify the tag to extract the values from
tag = 'venue_type'

# Initialize a dictionary to store unique values and their counts
unique_values = {}

# Extract unique values and count their occurrences
for item in json_data:
    value = item['venue_info'].get(tag)
    if value:
        if value not in unique_values:
            unique_values[value] = 1
        else:
            unique_values[value] += 1

# Print the unique values and their counts
for value, count in unique_values.items():
    print(f"{value}: {count}")


SHOPPING_CENTER: 9
APPAREL: 39
SHOPPING: 3
SUPERMARKET: 5
MARKET: 6
GROCERY: 21
CONVENIENCE_STORE: 1
GIFTS: 8
SOUVENIR_SHOP: 4


In [5]:
import spacy

# Load pre-trained word embeddings (e.g., spaCy's medium English model)
nlp = spacy.load("en_core_web_md")

# Given list of tags
tags = [
    "Parks",
    "Cultural Heritage",
    "Scenic Landmarks",
    "Nature Attractions",
    "Art",
    "Religious",
    "Library",
    "Shopping Center",
    "Fashion and Convenience",
    "Neighborhood Market",
    "Gifts & Souvenirs"
]

# User's input tag
user_input_tag = "Religious"

# Compute similarity between user's input tag and each tag from the list
similarities = []
user_input_tag_embedding = nlp(user_input_tag).vector
for tag in tags:
    tag_embedding = nlp(tag).vector
    similarity = user_input_tag_embedding.dot(tag_embedding) / (np.linalg.norm(user_input_tag_embedding) * np.linalg.norm(tag_embedding))
    similarities.append(similarity)

# Sort the similarities and retrieve the most similar tags
sorted_indices = np.argsort(similarities)[::-1]  # Descending order
most_similar_tags = [tags[i] for i in sorted_indices]

# Print the most similar tags
print("Most similar tags to '{}':".format(user_input_tag))
for tag in most_similar_tags:
    print(tag)


Most similar tags to 'Religious':
Religious
Cultural Heritage
Nature Attractions
Fashion and Convenience
Library
Neighborhood Market
Shopping Center
Art
Scenic Landmarks
Gifts & Souvenirs
Parks


In [2]:
pip install folium


     -------------------------------------- 102.3/102.3 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import folium
ssh_host = '137.43.49.79'
ssh_port = 22
ssh_username = 'student'
ssh_password = 'Ucd-cs-2023!'

mysql_user = 'root'
mysql_password = 'Group@18'
mysql_db = 'smartcityexplorer'

with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_password=ssh_password,
    remote_bind_address=('127.0.0.1', 3306)
) as tunnel:
    conn = pymysql.connect(
        host='127.0.0.1',
        user=mysql_user,
        password=mysql_password,
        db=mysql_db,
        port=tunnel.local_bind_port
    )
    
    cursor = conn.cursor()
    query = "SELECT name, zone_id, latitude, longitude FROM venue_static"
    data = pd.read_sql(query, conn)

In [4]:
# Set the initial map center and zoom level
map = folium.Map(location=[40.8042, -73.9915], zoom_start=12)


In [5]:
# Iterate over the table data and add markers to the map
for index, row in data.iterrows():
    if not pd.isna(row['longitude']) and not pd.isna(row['latitude']):
        folium.Marker([row['latitude'], row['longitude']], popup=row['zone_id']).add_to(map)


In [6]:
# Save the map to an HTML file
map.save('map.html')

# Display the map in the Jupyter Notebook or IDE
map
